In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv('./news/정상기업뉴스.csv')
data

,기업,기사발행일,기사제목,뉴스기사본문
0,씨제이이엔엠,2011.12.13. 오전 8:24,"씨제이이엔엠, 유형자산 취득 후 처분 예정",씨제이이엔엠은 1276억 7200만원 규모의 유형자산을 취득한다고 13일 공시했다....
1,씨제이이엔엠,2011.08.16. 오전 9:21,"코스닥, IT부품주 '훨훨'",16일 코스닥 시장에서는 IT부품주들이 강세를 보이고 있다. 특히 구글의 모토로라 ...
2,씨제이이엔엠,2011.06.08. 오전 7:58,장마감후 주요 종목뉴스-7일,[머니투데이 정영일 기자]◇코스피 시장\r\n△CJ는 법인청산에 따라 CJ푸드서비스...
3,씨제이이엔엠,2011.06.07. 오후 5:31,"CJ, CJ푸드서비스 계열사서 제외",[머니투데이 김선주 기자]CJ는 법인청산에 따라 CJ푸드서비스를 계열사에서 제외한다...
4,씨제이이엔엠,2011.11.17. 오전 8:47,시초가 소폭 하락 예상 - 코스닥 동시호가,17일 코스닥 시장은 소폭 하락 출발이 예상된다.\r\n\r\n코스닥 시장은 현재 ...
...,...,...,...,...
7373,에이치엘비,2020.12.04. 오후 4:15,"에스퓨얼셀, 에이치엘비와 수소선박 공동개발 '맞손'…""2023년 상용화 목표""",[한국경제TV 김정필 IT벤처팀장]\r\n\r\n\r\n-168·252kW급 수소연...
7374,에이치엘비,2020.12.14. 오전 9:39,"넥스트사이언스 ""나노젠, 코로나 백신 내년 5월 양산 계획""",[아시아경제 유현석 기자] “재조합 단백질 조각(Recombinant protein...
7375,에이치엘비,2020.12.30. 오후 5:45,"코스닥, 23년만에 최고치…시총도 지각변동","코스닥지수, 지난해 말 대비 44.6% 상승\r\n개인 비중 88.2%로 확대…거래..."
7376,에이치엘비,2020.12.10. 오후 7:21,제3회 강소기업대상 시상식서 에이치엘비파워 등 22개 기업 수상,한국강소기업협회가 주최한 제3회 대한민국 강소기업대상 시상식에서 에이치엘비파워 등 ...


In [3]:
data.groupby('기업').count()

,기사발행일,기사제목,뉴스기사본문
기업,,,
경창산업,69,69,69
고영테크놀러지,55,55,55
광림,55,55,55
금화피에스시,97,97,97
나스미디어,69,69,69
...,...,...,...
하이록코리아,73,73,73
한국정보통신,22,22,22
해성산업,25,25,25


In [5]:
# 제거할기사제목 = ['증시 일정', '증시일정', '장마감후', '장 마감 후', '장마감공시', '증시 캘린더', '재송', '투자정보', \
#     '코스닥 기업공시', '장중 주요', '코스닥 3분기 결산', '주요 뉴스 및 공시', '기업공시',' 기업 공시', '장 종료 후', \
#         '특징주A/S', '주요 정보', '오전 공시', '투자정보', '기업 공시', '오늘의 메모', '재테크 캘린더', '추천주 정리', '희망복원 주식클리닉', \
#         '\d{1,2}월 \d{1,2}일', '오늘의 주요 공시', '코스닥 공시', '코스닥 메모', '대박 공모주', '<표>', '오늘의 리포트', \
#         '공략주', '주식상담소', '주식왕 따라잡기', '주식컨설팅', '\d{4} 증시', '춤추는 테마주', '개장시황', \
#         '폭등신호 터진', '대폭등', '매드머니', '주담과 Q&A', '굿바이 \d{4}', '지금 당장 사라', '폭등주', \
#         '\d{4}%', '김정일 사망', '종목대탐험', '종목신호등', 'VIP CLUB', '수급유망주', '기상도', '중소형주', \
#         '국민주식고충처리반', '유망주', '머니Q', '시초가잡아라', '기관 Q&A', '부동산에 투자하려면', '베스트애널리스트', \
#         '코스닥협회장 취임', '코스닥협회', '부동산에 투자하는 방법', '국가품질 경영대회', '증시일정', '머니Q', '추천종목', '티타임 공략주', \
#         '수급유망주', '종목배틀', '\d{4} 증시 결산', '기업설명회', '종목신호등', 'VIP CLUB 추천주', '내일장 공략주 10선', '주간컨센서스동향', \
#         '주담과 Q&A', '조회공시', '주가급등 사유', '관련株들', '게임株', '이시각 Up&Down', '장내 매도', '[인사]', '증시기상도', \
#         '[재송]', '마감시황', '주식부자 속출', '교육주', '티타임 공략주', '\d{1,2}일 증권사 추천종목', '기업IR소식', '[승부주]']
# 제거제목리스트 = '|'.join(제거할기사제목)

In [31]:
제거할기사제목 = ['공략주', '특징주A/S', '교육주', '[인사]', '[승부주]', '[재송]']
제거제목리스트 = '|'.join(제거할기사제목)

In [32]:
data_unique = data.drop_duplicates(['뉴스기사본문'])
data_result = data_unique[data_unique['기사제목'].str.contains(제거제목리스트, na=False, case=False)]
data_result

,기업,기사발행일,기사제목,뉴스기사본문
3,씨제이이엔엠,2011.06.07. 오후 5:31,"CJ, CJ푸드서비스 계열사서 제외",[머니투데이 김선주 기자]CJ는 법인청산에 따라 CJ푸드서비스를 계열사에서 제외한다...
5,씨제이이엔엠,2011.03.12. 오전 10:01,"현대카드ㆍ현대캐피탈, 사업성공 프로그램 후원",현대카드·현대캐피탈은 성공을 향해 열정적으로 도전하는 개인사업자들의 이야기를 담은 ...
6,씨제이이엔엠,2011.03.11. 오후 4:21,"""27일까지 사업성공 스토리에 도전하세요""","[머니투데이 김유경 기자][현대카드·캐피탈, 우승자에게 3천만원 상당 인테리어 기부..."
9,안랩,2011.12.21. 오후 6:21,"안랩 ""김정일 사망 악용 악성코드 등장""",안철수 연구소는 북한 김정일 국방위원장 사망과 관련한 동영상 등을 통해 애드웨어 유...
12,포스코아이씨티,2011.11.14. 오전 9:32,"포스코아이씨티, 4분기부터 본격적인 이익증가 시현될 것","현대증권 한병화 애널리스트는 14일 포스코아이씨티에 대해 투자의견 매수, 적정주가 ..."
...,...,...,...,...
7366,에이치엘비,2020.09.23. 오후 3:07,"신한금융투자 ""에이치엘비, 변종·불법공매도 사실 아냐""","""시장조성자 역할 수행 과정에서 거래된 물량"" 주장\r\n관련 내용 제작·유포로 피..."
7370,에이치엘비,2020.12.30. 오전 11:16,"리보세라닙 병용 식도암 임상, 방사선 병행시 3명중 1명 `완전관해`",[이데일리 김재은 기자] 에이치엘비(028300)는 중국에서 수행된 리보세라닙(중국...
7371,에이치엘비,2020.12.14. 오전 9:59,"넥스트사이언스 관계사 나노젠 ""내년 5월 코로나백신 양산 계획""","[이데일리 김재은 기자] 넥스트사이언스(003580)는 관계사이자, 베트남 기업으로..."
7372,에이치엘비,2020.12.14. 오전 9:28,넥스트사이언스 관계사 나노젠 “코로나 백신 내년 5월 양산 계획”,사진=넥스트사이언스\r\n\r\n\r\n[파이낸셜뉴스] “재조합 단백질 조각(Rec...


In [25]:
pattern1 = r'\([^)]*\)'
pattern2 = r"\[([^]]+)\]"

data_result['뉴스기사본문전처리'] = [re.sub(pattern1, '', s) for s in data_result['뉴스기사본문']]
data_result['뉴스기사본문전처리'] = [re.sub(pattern2, '', s) for s in data_result['뉴스기사본문전처리']]
data_result['뉴스기사본문전처리'] = [re.sub('[^/^$|\s+/가-힣\.]', '', s) for s in data_result['뉴스기사본문전처리']]
data_result['뉴스기사본문전처리'] = [re.sub('[[ \s]{2,}\.{2,}]', '', s) for s in data_result['뉴스기사본문전처리']]
data_result['뉴스기사본문전처리'] = [re.sub('\n', '', s) for s in data_result['뉴스기사본문전처리']]
data_result['뉴스기사본문전처리'] = [re.sub('[/+]', '', s) for s in data_result['뉴스기사본문전처리']]
data_result['뉴스기사본문전처리'] = [re.sub('\r', '', s) for s in data_result['뉴스기사본문전처리']]
data_result.drop('뉴스기사본문', axis=1, inplace=True)
data_result
data_result.to_csv('dd.csv', index=None, encoding='utf-8-sig')

C:\Users\dbswo\AppData\Local\Temp\ipykernel_22100\1219599643.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_result['뉴스기사본문전처리'] = [re.sub(pattern1, '', s) for s in data_result['뉴스기사본문']]
C:\Users\dbswo\AppData\Local\Temp\ipykernel_22100\1219599643.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_result['뉴스기사본문전처리'] = [re.sub(pattern2, '', s) for s in data_result['뉴스기사본문전처리']]
C:\Users\dbswo\AppData\Local\Temp\ipykernel_22100\1219599643.py:6: SettingWithCopyWarning: 
A value is trying to be